In [8]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import cm
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures



filename = "AAPL.csv"
X, y = training_data_from_csv(filename_features="AAPL_X_learn.csv", 
        filename_variables="AAPL_y_learn.csv")
X = X.values # convert to numpy.ndarray used by sklearn
y = y.values # convert to numpy.ndarray used by sklearn
train_linear_reg(X, y)
# train_decision_tree(X, y, tscv, False)
# train_neural_net(X, y, tscv, False)
    
def training_data_from_csv(filename_features, filename_variables):
    """
    Load cvs file of features and target variables from csv
    Args:
        filename_features: string
        filename_variables: string
    Returns:
        pandas.DataFrame
    """
    X = pd.read_csv(filename_features, header=0)
    y = pd.read_csv(filename_variables, header=0)
    return X, y


def train_linear_reg(X, y):

    model = linear_model.LinearRegression()
    results = pd.DataFrame(columns=["final_train_idx", "MAE train", "MSE train", 
        "R2 train", "MAE test", "MSE test", "R2 test"])
    
    
    X_train = X[1000:]
    y_train = y[1000:]
    X_test = X[:1000]
    y_test = y[:1000]
    X_train = PolynomialFeatures(degree=2).fit(X_train).transform(X_train)
    X_test = PolynomialFeatures(degree=2).fit(X_test).transform(X_test)
    model.fit(X_train, y_train)
    # params = model.get_params()
    # coef = model.coef_
    y_pred = model.predict(X_train)
    mae_train, mse_train, r2_train = performance(y_train, y_pred)
    y_pred = model.predict(X_test)
    mae_test, mse_test, r2_test = performance(y_test, y_pred)
    results.loc[len(results)] = [train_idx[-1], mae_train, mse_train, r2_train,
            mae_test, mse_test, r2_test]
    plot_performance(results, "Linear Regression", "lin_reg") 
    
    

def performance(y_true, y_pred):
    """
    Calculates regression performance
    Args:
        y_true: numpy.ndarray
        y_pred: numpy.ndarray
    Returns:
        mae: mean_absolute_error
        mse: mean_squared_error
        r2: r2
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, r2

def plot_performance(results, title, filename):
    """
    Plot the results as subplots for each fold
    Args:
        results: pandas.DataFrame(columns=["fold", "MAE train", "MSE train", 
        "R2 train", "MAE test", "MSE test", "R2 test"])
        filename: str
    Returns:
        None
    """
    fig, axes = plt.subplots(3, 1, figsize=(10, 10))
    fig.tight_layout()
    plt.suptitle(title, fontsize=24)
    plt.subplots_adjust(left=0.1, top=0.9, right=0.9, bottom=0.1, hspace=0.6)
    results[["final_train_idx", "MAE train", "MAE test"]].plot(x="final_train_idx", kind="bar", 
        ax=axes[0], logy=True, colormap=cm.flag)
    axes[0].set_title("MAE")
    axes[0].set_xlabel("Final train index of fold")
    results[["final_train_idx", "MSE train", "MSE test"]].plot(x="final_train_idx", kind="bar", 
        ax=axes[1], logy=True, colormap=cm.flag)
    axes[1].set_title("MSE")
    axes[1].set_xlabel("Final train index of fold")
    results[["final_train_idx", "R2 train", "R2 test"]].plot(x="final_train_idx", kind="bar", 
        ax=axes[2], colormap=cm.flag)
    axes[2].set_title("R2")
    axes[2].set_xlabel("Final train index of fold")
    axes[2].set_ylim(0, 1)
    fig.savefig("images/{}.png".format(filename))

TypeError: train_linear_reg() takes exactly 3 arguments (2 given)